# Detecting arrythmia using Deep Leaning

## Loading modules

In [ ]:
!pip install matplotlib==3.1.3
!pip install wfdb wget

  Using cached matplotlib-3.1.3-cp37-cp37m-manylinux1_x86_64.whl (13.1 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.1
    Uninstalling matplotlib-3.5.1:
      Successfully uninstalled matplotlib-3.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
wfdb 3.4.1 requires matplotlib>=3.3.4, but you have matplotlib 3.1.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


  Using cached matplotlib-3.5.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.1.3
    Uninstalling matplotlib-3.1.3:
      Successfully uninstalled matplotlib-3.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import wfdb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv1D, LSTM, Dense, Dropout, TimeDistributed
# from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [ ]:
import matplotlib.pyplot as plt

## Download Dataset

In [ ]:
import wget
import zipfile

database_filename = "mit-bih-arrhythmia-database-1.0.0.zip"
database_path = Path(database_filename)

if not database_path.exists():
    url = f'https://physionet.org/static/published-projects/mitdb/mit-bih-arrhythmia-database-1.0.0.zip'
    wget.download(url)
    with zipfile.ZipFile(database_filename, 'r') as zip_ref:
        zip_ref.extractall(".")

## Loading data

Loading list of records available, from dataset available [here](https://physionet.org/content/mitdb/1.0.0/)

In [ ]:
records = np.loadtxt("mit-bih-arrhythmia-database-1.0.0/RECORDS", dtype=int)

Defining invalid beats as well as abnormal beats, according to [Physiobank](https://archive.physionet.org/physiobank/annotations.shtml)

In [ ]:
invalid_beat = [
    "[", "!", "]", "x", "(", ")", "p", "t", 
    "u", "`", "'", "^", "|", "~", "+", "s", 
    "T", "*", "D", "=", '"', "@"
]

abnormal_beats_dict = {
    0:"Normal beat",
    1:"Left bundle branch block beat",
    2:"Right bundle branch block beat",
    3:"Bundle branch block beat (unspecified)",
    4:"Atrial premature beat",
    5:"Aberrated atrial premature beat",
    6:"Nodal (junctional) premature beat",
    7:"Supraventricular premature or ectopic beat (atrial or nodal)",
    8:"Premature ventricular contraction",
    9:"R-on-T premature ventricular contraction",
    10:"Fusion of ventricular and normal beat",
    11:"Atrial escape beat",
    12:"Nodal (junctional) escape beat",
    13:"Supraventricular escape beat (atrial or nodal)",
    14:"Ventricular escape beat",
    15:"Paced beat",
    16:"Fusion of paced and normal beat",
    17:"Unclassifiable beat",
    18:"Beat not classified during learning",
    19:"Invalid beat"
}

abnormal_beats_index = {
    "N":0,
    "L":1,
    "R":2,
    "B":3,
    "A":4,
    "a":5,
    "J":6,
    "S":7,
    "V":8,
    "r":9,
    "F":10,
    "e":11,
    "j":12,
    "n":13,
    "E":14,
    "/":15,
    "f":16,
    "Q":17,
    "?":18,
}

## Processing dataset

This function classify a beat according to its symbol and the list provided above.

In [ ]:
def classify_beat(symbol):
    if symbol in abnormal_beats_index.keys():
        return int(abnormal_beats_index[symbol])
    elif symbol == ".":
        return 0
    else:
        return 19

Given a signal, the beat location, and the window to be used as a sequence, this function gets the sequence. It will return an empty array in case of an invalid beat or empty sequence.

In [ ]:
def get_sequence(signal, beat_loc, window_sec, fs):
    window_one_side = window_sec * fs
    beat_start = beat_loc - window_one_side
    beat_end = beat_loc + window_one_side
    if beat_end < signal.shape[0]:
        sequence = signal[beat_start:beat_end, 0]
        return sequence.reshape(1, -1, 1)
    else:
        return np.array([])

The code below will build a list of labels and sequences as well as map the sequences for each patient. The percentage calculated represents the ratio of abnormal beats in each patient data.

In [ ]:
all_sequences = []
all_labels = []
window_sec = 3
subject_map = []
for subject in records:
    record = wfdb.rdrecord(f'mit-bih-arrhythmia-database-1.0.0/{subject}')
    annotation = wfdb.rdann(f'mit-bih-arrhythmia-database-1.0.0/{subject}', 'atr')
    atr_symbol = annotation.symbol
    atr_sample = annotation.sample
    fs = record.fs
    scaler = StandardScaler()
    signal = scaler.fit_transform(record.p_signal)
    subject_labels = []
    for i, i_sample in enumerate(atr_sample):
        label = classify_beat(atr_symbol[i])
        sequence = get_sequence(signal, i_sample, window_sec, fs)
        if label is not None and sequence.size > 0:
            all_sequences.append(sequence)
            subject_labels.append(label)

    normal_percentage = sum(subject_labels) / len(subject_labels)
    subject_map.append({
        "subject": subject,
        "percentage": normal_percentage,
        "num_seq": len(subject_labels),
        "start": len(all_labels),
        "end": len(all_labels)+len(subject_labels)
    })
    all_labels.extend(subject_labels)

Creating bins to be used to stratify the train and validation split. 

In [ ]:
subject_map = pd.DataFrame(subject_map)

The code presented will create class in each patient is segmented.

In [ ]:
bins = [0, 0.2, 0.6, 1.0]
subject_map["bin"] = pd.cut(subject_map['percentage'], bins=3, labels=False, include_lowest=True)

In [ ]:
subject_map.head(10)

,subject,percentage,num_seq,start,end,bin
0,100,0.061837,2264,0,2264,0
1,101,0.106109,1866,2264,4130,0
2,102,14.339899,2183,4130,6313,2
3,103,0.058541,2084,6313,8397,0
4,104,14.377498,2302,8397,10699,2
5,105,0.989933,2682,10699,13381,0
6,106,2.610048,2090,13381,15471,0
7,107,14.810038,2132,15471,17603,2
8,108,0.728674,1817,17603,19420,0
9,109,1.126733,2525,19420,21945,0


Now, the dataset is split into train and validation, stratifying by the bin defined above.

In [ ]:
train, validation = train_test_split(subject_map, test_size=0.2, stratify=subject_map["bin"], random_state=42)

This function build a dataset based on the map for each split.

In [ ]:
def one_hot(a, num_classes):
  return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

def build_dataset(df, all_sequences, all_labels):
    sequences = []
    labels = []
    for i, row in df.iterrows():
        start = int(row["start"])
        end = int(row["end"])
        sequences.extend(all_sequences[start:end])
        labels.extend(all_labels[start:end])
        
    return np.vstack(sequences), np.vstack(one_hot(np.array(labels),20))

In [ ]:
X_train, y_train = build_dataset(train, all_sequences, all_labels)
X_val, y_val = build_dataset(validation, all_sequences, all_labels)

In [ ]:
X_train.shape

(89702, 2160, 1)

In [ ]:
X_train.shape, y_train.shape

((89702, 2160, 1), (89702, 20))

In [ ]:
def test_pipeline(subject):
    sequences = []
    labels = []
    window_sec = 3

    record = wfdb.rdrecord(f'mit-bih-arrhythmia-database-1.0.0/{subject}')
    annotation = wfdb.rdann(f'mit-bih-arrhythmia-database-1.0.0/{subject}', 'atr')
    atr_symbol = annotation.symbol
    atr_sample = annotation.sample
    
    fs = record.fs
    scaler = StandardScaler()
    signal = scaler.fit_transform(record.p_signal)
    
    for i, i_sample in enumerate(atr_sample):
        label = classify_beat(atr_symbol[i])
        sequence = get_sequence(signal, i_sample, window_sec, fs)
        if label is not None and sequence.size > 0:
            sequences.append(sequence)
            labels.append(label)

    X, y = np.vstack(sequences), np.vstack(one_hot(np.array(labels),20))
    print(X.shape,y.shape)
    return X, y

In [ ]:
X_test, y_test = test_pipeline(113)
print(X_test[0],y_test[0])

(1788, 2160, 1) (1788, 20)
[[ 0.13364177]
 [ 0.13364177]
 [ 0.097287  ]
 ...
 [-0.65404487]
 [-0.61769011]
 [-0.62980836]] [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## Training the model

### CNN model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, Flatten, Dense, Dropout
# from keras.optimizers import Adam

sequence_size = X_train.shape[1]
n_features = 1

cnn_model = Sequential([
    Conv1D(
        filters=8,
        kernel_size=4,
        strides=1,
        input_shape=(sequence_size, n_features),
        padding="same",
        activation="relu"
    ),
    Flatten(),
    Dropout(0.5),
    Dense(
        20,
        activation="sigmoid",
        name="output",
    )
])

optimizer = Adam(lr=0.001)
# Compiling the model
cnn_model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy"]
)
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2160, 8)           40        
                                                                 
 flatten (Flatten)           (None, 17280)             0         
                                                                 
 dropout (Dropout)           (None, 17280)             0         
                                                                 
 output (Dense)              (None, 20)                345620    
                                                                 
Total params: 345,660
Trainable params: 345,660
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

GPU found


In [ ]:
hist_cnn = cnn_model.fit(
    X_train, 
    y_train, 
    batch_size=128,
    epochs=15,
    validation_data=(X_val, y_val)
)

Epoch 1/15
701/701 [==============================] - 19s 12ms/step - loss: 0.0417 - accuracy: 0.8873 - val_loss: 0.1015 - val_accuracy: 0.7304
Epoch 2/15
701/701 [==============================] - 8s 12ms/step - loss: 0.0272 - accuracy: 0.9257 - val_loss: 0.1113 - val_accuracy: 0.7376
Epoch 3/15
701/701 [==============================] - 8s 11ms/step - loss: 0.0236 - accuracy: 0.9383 - val_loss: 0.1226 - val_accuracy: 0.7294
Epoch 4/15
701/701 [==============================] - 7s 11ms/step - loss: 0.0213 - accuracy: 0.9470 - val_loss: 0.1312 - val_accuracy: 0.7343
Epoch 5/15
701/701 [==============================] - 7s 11ms/step - loss: 0.0202 - accuracy: 0.9512 - val_loss: 0.1507 - val_accuracy: 0.7297
Epoch 6/15
701/701 [==============================] - 7s 11ms/step - loss: 0.0192 - accuracy: 0.9541 - val_loss: 0.1568 - val_accuracy: 0.7223
Epoch 7/15
701/701 [==============================] - 7s 10ms/step - loss: 0.0185 - accuracy: 0.9567 - val_loss: 0.1555 - val_accuracy: 0.733

In [ ]:
cnn_model.evaluate(X_val, y_val)

705/705 [==============================] - 3s 4ms/step - loss: 0.2025 - accuracy: 0.7233


[0.20253440737724304, 0.7232701778411865]

In [ ]:
cnn_model.evaluate(X_test, y_test)

56/56 [==============================] - 0s 4ms/step - loss: 0.0016 - accuracy: 0.9983


[0.0016043315408751369, 0.9983221292495728]

In [ ]:
np.unique(all_labels, return_counts=True)

(array([ 0,  1,  2,  4,  5,  6,  7,  8, 10, 11, 12, 14, 15, 16, 17, 19]),
 array([74795,  8052,  7235,  2536,   150,    83,     2,  7113,   801,
           16,   229,   106,  6999,   982,    33,  3101]))

In [ ]:
# summarize history for accuracy
plt.plot(hist_cnn.history['accuracy'])
plt.plot(hist_cnn.history['val_accuracy'])
plt.title('CNN model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
# summarize history for loss
plt.plot(hist_cnn.history['loss'])
plt.plot(hist_cnn.history['val_loss'])
plt.title('CNN model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
cnn_model.save("cnn_model.h5")
model = tf.keras.models.load_model("cnn_model.h5")
ypred = model.predict(np.array([X_test[20]]))
np.argmax(ypred)

4

### CNN with LSTM model

In [ ]:
sequence_size = X_train.shape[1]
n_features = 1 
n_subsequences = 4
subsequence_size = int(sequence_size / n_subsequences)

# Reshaping to be (samples, subsequences, sequence, feature)
X_train = X_train.reshape(-1, n_subsequences, subsequence_size, n_features)
X_val = X_val.reshape(-1, n_subsequences, subsequence_size, n_features)

In [ ]:
cnn_lstm_model = Sequential([
    TimeDistributed(
        Conv1D(
            filters=8,
            kernel_size=4,
            strides=1,
            padding="same",
            activation="relu"
        ), 
        input_shape=(n_subsequences, subsequence_size, n_features)
    ),
    TimeDistributed(Flatten()),
    LSTM(units=4),
    Dense(
        20,
        activation="sigmoid",
        name="output",
    )
])

optimizer = Adam(lr=0.001)
# Compiling the model
cnn_lstm_model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=["accuracy"]
)
cnn_lstm_model.summary()

In [ ]:
train_params = {
    "batch_size": 128,
    "epochs": 15,
    "verbose": 1,
    "validation_data": (X_val, y_val),
}

history_cnn_lstm = cnn_lstm_model.fit(X_train, y_train, **train_params)

In [ ]:
cnn_lstm_model.evaluate(X_val, y_val)

In [ ]:
# summarize history for accuracy
plt.plot(history_cnn_lstm.history['accuracy'])
plt.plot(history_cnn_lstm.history['val_accuracy'])
plt.title('CNN-LSTM model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history_cnn_lstm.history['loss'])
plt.plot(history_cnn_lstm.history['val_loss'])
plt.title('CNN-LSTM model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()